
### **Chapter 5.2: MPC for tracking task and Robuct MPC**



In this chapter, we explore and analyse practical implementations of Model Predictive Control (MPC) for reference tracking, and its extension to Robust MPC (RMPC) under uncertainty.

In the first part, we implement a basic tracking MPC, and then simulate and visualize its performance. Then we compare the tracking and stabilization MPC to highlight their structural differences and draw some key insights for hyperparemeter tuning.

In the second part, we extend the framework to Robust MPC: we first show how to implement a RMPC class, then visualize how we compute the robust invariant set in the phase portrait, and finally run full simulations to evaluate RMPC performance under uncertainty.


All the contents are summarized in the table below.  


<table border="1" style="border-collapse: collapse; text-align: center;">
  <!-- Title Row -->
  <tr>
    <th colspan="2" style="text-align:center">Content of Chapter 5.2 Exercise</th>
  </tr>

  <!-- Row group 1 -->
  <tr>
    <td rowspan="3">MPC for Reference Tracking</td>
    <td>Example 1: Tracking MPC Implementation</td>
  </tr>
  <tr>
    <td>Example 2: Simulation and Visualization</td>
  </tr>
  <tr>
    <td>Example 3: MPC for Stabilization vs. MPC for Tracking</td>
  </tr>

  <!-- Row group 2 -->
  <tr>
    <td rowspan="3">Robust MPC (RMPC)</td>
    <td>Example 1: RMPC Implementation</td>
  </tr>
  <tr>
    <td>Example 2: Robust Invariant Set and Phase Portrait for RMPC</td>
  </tr>
  <tr>
    <td>Example 3: Simulation and Visualization</td>
  </tr>

</table>

First, we need to set up our Python environment and specify some scenarios as a testbed for the following examples.

In [ ]:
import sys
import os
sys.path.append(os.path.abspath(".."))
from rest.utils import *

import numpy as np
import casadi as ca

import pytope as pt
from scipy.spatial import ConvexHull, QhullError

In [ ]:
# Define profile of slope, the initial / target state
case = 1 # 1 or 2 or 3 or 4
initial_position = -0.5
initial_velocity = 0.0
target_position = 0.6
target_velocity = 0.0

# Define the constraints for input (potentially used)
input_lbs = -1.0
input_ubs = 1.0

# Case 1: unconstrained case
env = Env(case, np.array([initial_position, initial_velocity]), np.array([target_position, target_velocity]))
dynamics = Dynamics(env)

# Case 2: constrained case
env_constr = Env(case, np.array([initial_position, initial_velocity]), np.array([target_position, target_velocity]),
                 input_lbs=input_lbs, input_ubs=input_ubs)
dynamics_constr = Dynamics(env_constr)


<br>

----

### **Part (a): MPC for Reference Tracking**

So far we have limited our discussion on MPC to the case of stabilization. However, in many applications, we want our system to track a given trajectory instead of stabilizing the system at the origin. In the following, we will briefly look at the case of tracking a reference trajectory using MPC. 

Let the trajectory be defined as $\boldsymbol{r}_k \in \mathbb{R}^s, \forall k \in \{ 0, \dots, T + N \}$ with $s \leq n$ and $T > 0$ and let the output of the system be given by


$$
\boldsymbol{y_k} = \boldsymbol{C} \boldsymbol{x_k} \in \mathbb{R}^s \,,
$$

where $C \in \mathbb{R}^{s \times n}$ maps the full state vector $\boldsymbol{x_k}$ to the output $\boldsymbol{y_k}$, which only consists of the elements of the state vector that we want to track. 
Then the open-loop optimization for the tracking MPC with quadratic tracking objective is:

$$
J_k^*(\boldsymbol{x_k}) = \min_{u_{k|k}, \ldots, u_{k+N-1|k}} \sum_{i=0}^{N-1} \boxed{ \left( \boldsymbol{y_{k+i|k}} - \boldsymbol{r_{k+i|k}} \right)^T \boldsymbol{Q} \left( \boldsymbol{y_{k+i|k}} - \boldsymbol{r_{k+i|k}} \right) } + u_{k+i|k} ^T \boldsymbol{R} u_{k+i|k}
$$

subject to:

$$
\boldsymbol{x_{k+i+1|k}} = \boldsymbol{f} \left( \boldsymbol{x_{k+i|k}}, \boldsymbol{u_{k+i|k}} \right), \quad \forall i \in \{0, \ldots, N-1\},
$$

$$
\boldsymbol{x_{k+i|k}} \in \mathcal{X}, \quad \forall i \in \{0, \ldots, N\},
$$

$$
u_{k+i|k} \in \mathcal{U}, \quad \forall i \in \{0, \ldots, N-1\},
$$

$$
\boldsymbol{x_{k+N|k}} \in \mathcal{X}_f, 
$$

$$
\boxed{\boldsymbol{y_{k+i|k}} = \boldsymbol{C} \boldsymbol{x_{k+i|k}}, \quad \forall i \in \{0, \ldots, N\},}
$$

$$
\boldsymbol{x_{k|k}} = \boldsymbol{\overline{x}_{k}}
$$

where $Q$ and $R$ are positive semidefinite and positive definite weighting matrices, respectively. Using an MPC has the advantage that we can apply control inputs to the system that anticipate changes in the reference. Furthermore, MPC allows us to enforce state and input constraints during the tracking task. Using a similar objective function we can also track a control input trajectory while satisfying state and input constraints. 




<br>

#### **Example 1: Tracking MPC Implementation**

Based on the tracking MPC formulation introduced earlier, we now present the corresponding controller implementation. Compared to the MPC for stabilization tasks, the tracking MPC shares the same structural formulation of the optimization problem. As a result, the `setup` function does not require any modification. In the following, we provide only the updated control loop to reflect the specific problem setting of tracking MPC. In each cycle, you need to:

1\) Assign values for the initial state (`lbx` and `ubx` for $i = 0$) via the interface `set()`;  

2\) Assign values for the tracking reference (`yref` for $i \in \{ 1, ... , N \}$) via the interface `set()`;  

&nbsp;&nbsp;&nbsp;&nbsp;*Note that: the reference state here is a time-varying array, you need to check the value from variable `self.traj_ref`.*

3\) Solve OCP problem by calling `solve()`;  

4\) Get the first input command through `get()`;  

In [ ]:

def compute_action_external(self, current_state: np.ndarray, current_time) -> np.ndarray:
    """
    Solve the MPC problem and compute the optimal control action.

    Args:
    - current_state: The current state of the system.
    - current_time: The current time (not used in this time-invariant case).

    Returns:
    - Optimal control action.
    """

    # Update initial state in the solver
    self.solver.set(0, "lbx", current_state)
    self.solver.set(0, "ubx", current_state)

    # Update reference trajectory for all prediction steps
    input_ref = np.zeros(self.dim_inputs)
    ref_length = self.traj_ref.shape[0]

    for i in range(self.N):
        index = min(current_time + i, ref_length - 1)
        state_ref = self.traj_ref[index, :self.dim_states]
        self.solver.set(i, "yref", np.concatenate((state_ref, input_ref)))
    index = min(current_time + self.N, ref_length - 1)
    terminal_state_ref = self.traj_ref[index, :self.dim_states]
    self.solver.set(self.N, "yref", terminal_state_ref)

    # Solve the MPC problem
    status = self.solver.solve()
    #if status != 0:
    #    raise ValueError(f"Acados solver failed with status {status}")

    # Extract the first control action
    u_optimal = self.solver.get(0, "u")

    # Extract the predictions
    x_pred = np.zeros((self.N + 1, self.dim_states))
    u_pred = np.zeros((self.N, self.dim_inputs))
    for i in range(self.N + 1):
        x_pred[i, :] = self.solver.get(i, "x")
        if i < self.N:
            u_pred[i, :] = self.solver.get(i, "u")

    if self.verbose:
        print(f"Optimal control action: {u_optimal}")
        print(f"x_pred: {x_pred}")
        print(f"u_pred: {u_pred}")

    return u_optimal, x_pred, u_pred


<br>

#### **Example 2: Simulation and Visualization**  

1\) Subtitute the function `compute_action` in predefined class `TrackingMPCController` with our external control loop `compute_action_external`;

2\) Specify the arguments as follows:  

- Parameters:  

    i) weight for state $\bm{Q} = \text{diag}([1, 1])$ (requirement: symmetric, positive semi-definite matrix)  

    ii) weight for input $\bm{R} = [0.1]$ (requirement: symmetric, positive definite matrix)  
    
    iii) control frequency $f = 20$

    iv) horizon $N = 20$

3\) Instantiate the controller `LQRController` and run the simulation to get generate a reference trajectory `state_traj_lqr`;

4\) Use the specified hyperparameetrs and the reference trajectory from LQR to instantiate the controller class `TrackingMPCController`;

5\) Instantiate the class `Simulator` and call function `run_simulation()` to generate the simulated state- and input-trajectory;

6\) Instantiate the class `Visualizer`, call function `display_final_results()` and `display_animation()` to show the simulations;

In [ ]:
TrackingMPCController.compute_action = compute_action_external

In [ ]:
# Define weight matrix in stage and terminal cost and the horizon for MPC 
Q = np.diag([1, 1])
R = np.array([[0.1]])
Qf = Q
N = 20

# Define parameters of simulation
freq = 20 # controll frequency
t_terminal = 8 # time length of simulation

In [ ]:
# Define the LQR controller
controller_lqr = LQRController(env_constr, dynamics_constr, Q, R, freq)

# Instantiate the simulator, run the simulation, and plot the results
simulator_lqr = Simulator(dynamics_constr, controller_lqr, env_constr, 1/freq, t_terminal)
simulator_lqr.run_simulation()
state_traj_lqr, input_traj_lqr = simulator_lqr.get_trajectories()

In [ ]:
# Define the MPC controller
controller_mpc = TrackingMPCController(env_constr, dynamics_constr, Q, R, Qf, freq, N, traj_ref=state_traj_lqr, name='tracking_MPC', verbose=False)

# Instantiate the simulator, run the simulation, and plot the results
simulator_mpc = Simulator(dynamics_constr, controller_mpc, env_constr, 1/freq, t_terminal)
simulator_mpc.run_simulation()

# Instantiate the visualizer, and display the plottings and animation
visualizer_mpc = Visualizer(simulator_mpc)
visualizer_mpc.display_plots("Example of Tracking MPC with LQR Reference Trajectory")
visualizer_mpc.display_animation()

<br>

#### **Example 3: MPC for Stabilization vs. MPC for Tracking**

After implementing the tracking MPC controller, we now return to one of the questions raised at the very beginning: Which hyperparameters affect MPC performance, and in what way? In Part 1, we have already discussed this question in the context of MPC for stabilization. The most influential hyperparameters include:

- The relation between state cost $\boldsymbol{Q}$ and input cost $\boldsymbol{R}$: A larger $\boldsymbol{Q}/\boldsymbol{R}$ ratio typically leads to more aggressive tracking behavior, prioritizing state accuracy over input efficiency. Conversely, higher input penalties result in smoother but slower responses.

- The horizon length $N$: Increasing $N$ allows the controller to better foresee long-term consequences, often improving performance at the cost of computational burden. A short horizon can lead to shortsighted behavior and suboptimal transients.

- Whether we introduce the special terminal cost design $g_N(\boldsymbol{x}_{k+N|k})$: An appropriately designed terminal cost improves closed-loop behavior and convergence properties, especially in stabilization problems. 


In [ ]:
# Define the MPC controller (reference)
controller_mpc_ref = LinearMPCController(env_constr, dynamics_constr, Q, R, Qf, freq, N, name='MPC_N=20_Q=1(baseline)', verbose=False)

# Instantiate the simulator, run the simulation, and plot the results
simulator_mpc_ref = Simulator(dynamics_constr, controller_mpc_ref, env_constr, 1/freq, t_terminal)
simulator_mpc_ref.run_simulation()

In [ ]:
## Example 1: larger Q value
Q_larger = np.diag([100, 100])
Qf = Q

# Define the MPC controller
controller_mpc = LinearMPCController(env_constr, dynamics_constr, Q_larger, R, Qf, freq, N, name='MPC_N=20_Q=100', verbose=False)

# Instantiate the simulator, run the simulation, and plot the results
simulator_mpc = Simulator(dynamics_constr, controller_mpc, env_constr, 1/freq, t_terminal)
simulator_mpc.run_simulation()

# Instantiate the visualizer, and display the plottings and animation
visualizer_mpc = Visualizer(simulator_mpc)
visualizer_mpc.display_contrast_plots("Impact of tuning matrix Q (stabilization task)", simulator_mpc_ref)
visualizer_mpc.display_contrast_animation_same(simulator_mpc_ref)

In [ ]:
## Example 2: larger N 
N_test = 60

# Define the MPC controller
controller_mpc = LinearMPCController(env_constr, dynamics_constr, Q, R, Qf, freq, N_test, name='MPC_N=60_Q=1', verbose=False)

# Instantiate the simulator, run the simulation, and plot the results
simulator_mpc = Simulator(dynamics_constr, controller_mpc, env_constr, 1/freq, t_terminal)
simulator_mpc.run_simulation()

# Instantiate the visualizer, and display the plottings and animation
visualizer_mpc = Visualizer(simulator_mpc)
visualizer_mpc.display_contrast_plots("Impact of raising prediction horizon N (stabilization task)", simulator_mpc_ref)
visualizer_mpc.display_contrast_animation_same(simulator_mpc_ref)

In [ ]:
## Example 3: use DARE to compute the infinite LQR weight matrix as terminal cost in MPC

# Linearize dynamics at equilibrium and solve DARE to compute the infinite LQR weight matrix
A_d, B_d = dynamics.get_linearized_AB_discrete(np.array([0.0, 0.0]), 0.0, 1/freq)
P = scipy.linalg.solve_discrete_are(A_d, B_d, Q, R)

# Define the MPC controller
controller_mpc = LinearMPCController(env_constr, dynamics_constr, Q, R, P, freq, N, name='MPC_with_terminal_cost', verbose=False)

# Instantiate the simulator, run the simulation, and plot the results
simulator_mpc = Simulator(dynamics_constr, controller_mpc, env_constr, 1/freq, t_terminal)
simulator_mpc.run_simulation()

# Instantiate the visualizer, and display the plottings and animation
visualizer_mpc = Visualizer(simulator_mpc)
visualizer_mpc.display_contrast_plots("Impact of introducing infinite LQR cost as terminal cost (stabilization task)", simulator_mpc_ref)
visualizer_mpc.display_contrast_animation_same(simulator_mpc_ref)

From the simulation results of Example 2 and Example 3, we observe that the resulting trajectories are quite similar. Is there a deeper connection between improving MPC performance by extending the prediction horizon $N$ and by introducing a terminal cost $g_N(\boldsymbol{x}_{k+N|k})$?

<blockquote style="padding-top: 20px; padding-bottom: 10px;">

##### **💡 Take-away 1: Key Insight on the Role of Terminal Cost $g_N(\boldsymbol{x}_{k+N|k})$ in MPC**

**Adding a terminal cost can be understood as a structured way of extending the prediction horizon without increasing the number of optimization variables**. While increasing the horizon allows the controller to explicitly consider longer-term consequences, introducing a terminal cost implicitly achieves a similar effect by assigning a cost-to-go approximation to the terminal state. In other words, $g_N(\boldsymbol{x}_{k+N|k})$ serves as a surrogate for the cost that would be accumulated beyond the finite horizon.  

Thus, terminal cost acts as a low-computation alternative to horizon extension, offering similar benefits in convergence speed and policy aggressiveness—provided that the terminal cost is well designed.


<br>

The previous section demonstrated how modifying three key hyperparameters affected controller performance in the stabilization task. As we can observe, compared to tuning the weights $\boldsymbol{Q}$ and $\boldsymbol{R}$, increasing the prediction horizon or adding terminal components tends to have a more pronounced impact on the overall system performance. More specifically, extending the horizon improves long-term behavior but comes at the cost of increased computational complexity. In contrast, adding terminal components typically incurs negligible additional computation time, but it requires a reasonable estimate of the terminal cost function.
The previous section demonstrated how modifying three key hyperparameters affected controller performance in the stabilization task. As we can observe, compared to tuning the weights $\boldsymbol{Q}$ and $\boldsymbol{R}$, increasing the prediction horizon or adding terminal components tends to have a more pronounced impact on the overall system performance. More specifically, extending the horizon improves long-term behavior but comes at the cost of increased computational complexity. In contrast, adding terminal components typically incurs negligible additional computation time, but it requires a reasonable estimate of the terminal cost function.

We conclude these properties as the tuning principle follows:


<blockquote style="padding-top: 20px; padding-bottom: 10px;">

##### **💡 Take-away 2: Principles for Tuning MPC in Stabilization Tasks**

If you aim to improve the performance of your MPC controller in a stabilization task, consider the following principles:

- **If you can afford increased computational load**, extending the prediction horizon $N$ is often the most effective way to enhance long-term planning and convergence behavior.

- **OR if you have a reasonable approximation of the cost-to-go or insight into the terminal state distribution**, adding a well-designed terminal cost can offer similar benefits to increasing the horizon, often with much lower computational cost.

- **If neither of the above is feasible**, tuning the cost weights $\boldsymbol{Q}$ and $\boldsymbol{R}$ can still improve controller behavior, though the resulting performance gains are typically smaller and more nuanced.

<br>

##### **MPC for Reference Tracking**

The previous section demonstrated how modifying three key hyperparameters affected controller performance in the stabilization task. As we can observe, compared to tuning the weights $\boldsymbol{Q}$ and $\boldsymbol{R}$, increasing the prediction horizon or adding terminal components tends to have a more pronounced impact on the overall system performance. More specifically, extending the horizon improves long-term behavior but comes at the cost of increased computational complexity. In contrast, adding terminal components typically incurs negligible additional computation time, but it requires a reasonable estimate of the terminal cost function.
However, in the context of **tracking task**, the influence of these hyperparameters differs, which will be illustrated in the following example:

In [ ]:
# Define the LQR controller
controller_lqr = LQRController(env_constr, dynamics_constr, Q, R, freq)

# Instantiate the simulator, run the simulation, and plot the results
simulator_lqr = Simulator(dynamics_constr, controller_lqr, env_constr, 1/freq, t_terminal)
simulator_lqr.run_simulation()
state_traj_lqr, input_traj_lqr = simulator_lqr.get_trajectories()



# Define the MPC controller
controller_mpc_ref = TrackingMPCController(env_constr, dynamics_constr, Q, R, Qf, freq, N, traj_ref=state_traj_lqr, name='tracking_MPC_baseline', verbose=False)

# Instantiate the simulator, run the simulation, and plot the results
simulator_mpc_ref = Simulator(dynamics_constr, controller_mpc_ref, env_constr, 1/freq, t_terminal)
simulator_mpc_ref.run_simulation()



# Linearize dynamics at equilibrium and solve DARE to compute the infinite LQR weight matrix
A_d, B_d = dynamics.get_linearized_AB_discrete(np.array([0.0, 0.0]), 0.0, 1/freq)
P = scipy.linalg.solve_discrete_are(A_d, B_d, Q, R)

# Define the MPC controller
controller_mpc_terminal = TrackingMPCController(env_constr, dynamics_constr, Q, R, P, freq, N, traj_ref=state_traj_lqr, name='tracking_MPC_with_terminal_cost', verbose=False)

# Instantiate the simulator, run the simulation, and plot the results
simulator_mpc_terminal = Simulator(dynamics_constr, controller_mpc_terminal, env_constr, 1/freq, t_terminal)
simulator_mpc_terminal.run_simulation()

# Instantiate the visualizer, and display the plottings and animation
visualizer_mpc_terminal = Visualizer(simulator_mpc_terminal)
visualizer_mpc_terminal.display_contrast_plots("Impact of introducing infinite LQR cost as terminal cost (tracking task)", simulator_mpc_ref)
visualizer_mpc_terminal.display_contrast_animation_same(simulator_mpc_ref)

<blockquote style="padding-top: 20px; padding-bottom: 10px;">

##### **🔍 Hands-on Exploration: Try to tune $\boldsymbol{Q}$ or raise the horizon $N$**

Here we only show the impact of introducing terminal cost, you may also want to try the impact of varying the value of $\boldsymbol{Q}$ or horizon $N$, what will happen then?


#### **Results Analysis**

From the simulation results, we observe that increasing the prediction horizon $N$ or adding a terminal cost has only a marginal effect on the performance of tracking MPC.

This is because in the tracking MPC cost function, the state tracking error is relatively evenly distributed across the prediction horizon. This implies that whether or not a larger horizon is used to capture more future states, the optimizer is already receiving consistent guidance from the reference trajectory throughout the horizon. As a result, the controller does not benefit significantly from longer lookahead or terminal extrapolation, unlike in stabilization tasks where future behavior must be inferred more structurally through horizon length and terminal shaping.

**In other words, if the reference trajectory is feasible, it already embeds information about the system's underlying dynamics—specifically, in the way the state evolves over time.** In such cases, the controller does not need to rely entirely on a long prediction horizon to infer the desired system behavior. **The trajectory itself provides a sufficient guidence that uniquely defining not only the steady-state, but also the transient-state,** thereby reducing the marginal benefit of increasing the horizon length purely for predictive accuracy.


<blockquote style="padding-top: 20px; padding-bottom: 10px;">

##### **💡 Take-away 3: Tuning MPC in Tracking Tasks**

**When tuning a tracking MPC controller for a feasible reference trajectory, the impact of hyperparameter choices tends to be significantly smaller than in stabilization tasks.** This is because the reference trajectory plays a dominant role in guiding the state evolution during optimization, thereby diminishing the influence of cost weights, horizon length, and terminal cost design.

**Although this method demands (Tracking MPC) some offline computation for computing the reference trajectory, its major advantage lies in the fact that tuning is considerably simpler and less sensitive to task-specific configurations, making it the recommended choice in many applications.**


<br>

----

### **Part (b): Robust MPC**

So far, we have considered MPC formulations under the assumption that the system dynamics are perfectly known and that there are no disturbances affecting the system. However, in many real-world applications, such assumptions rarely hold: model uncertainties, external disturbances, and unmodeled dynamics often degrade controller performance or even cause constraint violations. To address these challenges, we now turn to Robust Model Predictive Control (Robust MPC) — a class of methods designed to ensure constraint satisfaction and stability even in the presence of bounded disturbances or model-mismatches.

Robust MPC encompasses a variety of formulations, each differing in how they handle uncertainty and guarantee robustness. These include min-max MPC, scenario-based MPC, and tube-based MPC, among others. In this section, we focus on the tube-based approach, which is conceptually simple yet powerful enough to handle bounded additive disturbances. It works by planning a nominal trajectory and ensuring that the true system state remains within a bounded tube around it, using a combination of feedback and constraint tightening.

##### **Derivation of Error Dynamics:**

Given a linear time-invariant system with bounded additive disturbances:

$$
\boldsymbol{x}_{k+1} = \boldsymbol{A}\boldsymbol{x}_k + \boldsymbol{B}\boldsymbol{u}_k + \boldsymbol{w}_k,
$$

with $\boldsymbol{w}_k \in \mathcal{D}$, where $\mathcal{D}$ is compact and contains the origin.  We can construct an associated linear nominal model accordingly:

$$
\boldsymbol{\overline{x}}_{k+1} = \boldsymbol{A}\boldsymbol{\overline{x}}_k + \boldsymbol{B}\boldsymbol{\overline{u}}_k,
$$

Then the error between the true system state and the nominal system state is $\boldsymbol{e}_k = \boldsymbol{x}_k - \boldsymbol{\overline{x}}_k$. Using the pre-stabilization controller $u_k = \overline{u}_k + \boldsymbol{K} ( \boldsymbol{x}_k - \boldsymbol{\overline{x}}_k )$, the error dynamics can be derived as:

$$
\boldsymbol{e}_{k+1} = \underbrace{(\boldsymbol{A} + \boldsymbol{B} \boldsymbol{K})}_{\boldsymbol{A}_{\text{cl}}} \boldsymbol{e}_{k} + \boldsymbol{w}_{k}
$$

##### **Bounds for the Error: Robust Invariant Set $\Omega_{\text{tube}}$**

While $\boldsymbol{A}_{\text{cl}}$ being Schur stable (eigenvalues have norm less than $1$), this yields bounds on all future errors and will converge to an error set that we denote $\Omega_{\text{tube}}$. The error set $\Omega_{\text{tube}}$ can be naively (there exist more efficient computations) determined using recursion. With $\boldsymbol{e}_0 = \boldsymbol{0}$ this yields $\boldsymbol{e}_1 = \boldsymbol{w}_0 \in \mathcal{D}$. Therefore, the error set at the first time step is $\Omega_1 = \mathcal{D}$. Then the error set at the next time step can be computed by applying the error dynamics $\boldsymbol{A}_{\text{cl}}$ to every element of $\Omega_1$ and then expanding the resulting set of errors with every possible disturbance $\boldsymbol{w}_1 \in \mathcal{D}$. We continue this process until convergence. The recursion is given by the following Algorithm, where $\oplus$ denotes the Minkowski addition.


$$
\begin{array}{l}
\textbf{Algorithm: Computing the tube } \Omega_{\text{tube}} 
\end{array}
$$

$$
\begin{array}{l}
\text{Initialization: } A_{\text{cl}},\ \mathcal{D} \\
\text{01: } \quad k \gets 1 \\
\text{02: } \quad \Omega_1 \gets \mathcal{D} \\
\text{03: } \quad \textbf{while True do} \\
\text{04: } \quad | \quad \Omega_{k+1} \gets A_{\text{cl}} \Omega_k \oplus \mathcal{D} \\
\text{05: } \quad | \quad \textbf{if } \Omega_{k+1} = \Omega_k \textbf{ then} \\
\text{06: } \quad | \quad | \quad \Omega_{\text{tube}} \gets \Omega_k \\
\text{07: } \quad | \quad | \quad \textbf{Stop} \\
\text{08: } \quad | \quad \textbf{end if} \\
\text{09: } \quad | \quad k \gets k + 1 \\
\text{10: } \quad \textbf{end while}
\end{array}
$$


##### **Formulation of RMPC Problem:**

For the nominal model, tube-based MPC solves the following constrained optimization problem at every time step k to obtain the optimal sequence $\{\overline{u}_{k|k}, \ldots, \overline{u}_{k+N-1|k}\}$:

$$
J_k^*(\boldsymbol{\overline{x}_k}) = \min_{\overline{u}_{k|k}, \ldots, \overline{u}_{k+N-1|k}} 
g_N\left( \boldsymbol{\overline{x}_{k+N|k}} \right) + \sum_{i=0}^{N-1} g_i\left( \boldsymbol{\overline{x}_{k+i|k}}, \boldsymbol{\overline{u}_{k+i|k}} \right)
$$

subject to:

$$
\boldsymbol{\overline{x}_{k+i+1|k}} = \boldsymbol{A} \boldsymbol{\overline{x}_{k+i|k}} + \boldsymbol{B} \boldsymbol{\overline{u}_{k+i|k}}, \quad \forall i \in \{0, \ldots, N-1\},
$$

$$
\boldsymbol{\overline{x}_{k+i|k}} \in \mathcal{X} \ominus \mathcal{\Omega}_{\text{tube}}, \quad \forall i \in \{0, \ldots, N\},
$$

$$
\overline{u}_{k+i|k} \in \mathcal{U} \ominus \boldsymbol{K} \mathcal{\Omega}_{\text{tube}}, \quad \forall i \in \{0, \ldots, N-1\},
$$

$$
\boldsymbol{\overline{x}_{k+N|k}} \in \mathcal{X}_{f,\text{robust}}, 
$$

$$
\boldsymbol{x_k} \in \boldsymbol{\overline{x}_{k|k}} \oplus \mathcal{\Omega}_{\text{tube}},
$$

where $\boldsymbol{\overline{x}_{k+i|k}}$ is the open-loop nominal state at time step $k + i$, and $\mathcal{X} \ominus \mathcal{\Omega}_{tube} = \{\boldsymbol{x} \in R^n: \boldsymbol{x} + \boldsymbol{w} \in \mathcal{X}, \boldsymbol{w} \in \mathcal{\Omega}_{\text{tube}} \}$ and $\mathcal{U} \ominus \boldsymbol{K} \mathcal{\Omega}_{\text{tube}} = \{ u \in R^m : u + \boldsymbol{K} \boldsymbol{w} \in \mathcal{U}, \boldsymbol{w} ∈ \mathcal{\Omega}_{\text{tube}} \}$ are the state and input constraints tightened using the bounded tube Ωtube with $\ominus$ denoting the Pontryagin difference. 


In each step of RMPC, we solve the above optimization problem to obtain the nominal state and input trajectories. Then, by combining these nominal trajectories with a pre-calculated stabilizing feedback control gain, the overall RMPC policy can be expressed as:

$$
u_k = \overline{u}_k + \boldsymbol{K} ( \boldsymbol{x}_k - \boldsymbol{\overline{x}}_k )
$$

<br>

#### **Example 1: RMPC Implementation**  

In this section, we introduce how to implement an RMPC controller class. As before, we use `Acados` as the underlying solver. However, prior to calling the solver, we need to recursively compute the Robust Invariant Set starting from the disturbance set $\mathcal{D}$. For this step, we use the `pytopes` package, which provides basic polytope operations such as **affine mapping** and **Minkowski sums**. The implementation will follow three key steps:

1\) Complete the functions `affine_map` and `compute_invariant_tube` to compute the Robust Invariant Set $\Omega_{\text{tube}}$;  
    Hint: you may use the **affine mapping** and **Minkowski sums** operation from package `pytopes`, please refer to [website](https://github.com/heirung/pytope)

2\) Finish the solver configuration function `setup`, which is almost the same as for the nominal MPC implementation, but additionally requires interfaces for polytopic constraints;  
    Hint: here you can directly use the halfspace representation of the polytopes, which can be easily obtained by `pt.Polytope.A` and `pt.Polytope.b`

3\) Finish the control loop function `compute_action`, where two main differences exist:

&nbsp;&nbsp;&nbsp;&nbsp; i) set the upper limit of the shifted polytope constraint by $\boldsymbol{b}_{\text{current}} = \boldsymbol{A}_0 \boldsymbol{x}_{\text{current}} + \boldsymbol{b}_0$;

&nbsp;&nbsp;&nbsp;&nbsp; ii) after computing the nominal state $\boldsymbol{x}_k$ and input $\overline{u}_k$ from the solver, apply the stabilizing feedback policy $u_k = \overline{u}_k + \boldsymbol{K} ( \boldsymbol{x}_k - \boldsymbol{\overline{x}}_k )$;

In [ ]:
# Derived class for Tube-based Robust MPC Controller
class LinearRMPCController(LQRController):
    def __init__(
            self, 
            env: Env, 
            dynamics: Dynamics, 
            Q: np.ndarray, 
            R: np.ndarray, 
            Qf: np.ndarray, 
            freq: float, 
            N: int, 
            K_feedback: Optional[np.ndarray] = None,  # Feedback gain for tube
            disturbance_bounds: Optional[Tuple[np.ndarray, np.ndarray]] = None,  # (lbz, ubz) of D
            max_iter: int = 10,  # Max iterations for invariant set computation
            name: str = 'RMPC', 
            type: str = 'RMPC', 
            verbose: bool = True
        ) -> None:

        self.Qf = Qf

        self.N = N  # Prediction horizon

        self.ocp = None
        self.solver = None

        super().__init__(env, dynamics, Q, R, freq, name, type, verbose)

        x0 = np.zeros(self.dynamics.dim_states)
        u0 = np.zeros(self.dynamics.dim_inputs)
        self.A, self.B = self.dynamics.get_linearized_AB_discrete(x0, u0, self.dt)

        # Automatically solve DARE if no K provided
        if K_feedback is None:
            from scipy.linalg import solve_discrete_are
            P = solve_discrete_are(self.A, self.B, Q, R)
            self.K_feedback = -np.linalg.inv(R + self.B.T @ P @ self.B) @ self.B.T @ P @ self.A
        else:
            self.K_feedback = K_feedback

        # Compute Omega_tube from disturbance box D 
        if disturbance_bounds is not None:

            disturbance_lbs = disturbance_bounds[0]
            disturbance_ubs = disturbance_bounds[1]

        elif self.env.disturbance_lbs is not None and self.env.disturbance_ubs is not None:

            disturbance_lbs = self.env.disturbance_lbs
            disturbance_ubs = self.env.disturbance_ubs
            
        else:
            raise ValueError("No bounds of additive disturbances provided, can not initialize RMPC")
        
        disturbance_lbs /= freq
        disturbance_ubs /= freq
        
        self.Omega_tube = self.compute_invariant_tube(self.A + self.B @ self.K_feedback, disturbance_lbs, disturbance_ubs, max_iter=max_iter)

        # Create polytope for tighten state constraints
        dim = len(self.env.state_lbs)
        H_box = np.vstack([np.eye(dim), -np.eye(dim)])
        h_box = np.hstack([self.env.state_ubs, -self.env.state_lbs])
        self.X = pt.Polytope(H_box, h_box)
        self.X_tighten = self.X - self.Omega_tube

        # Create polytope for tighten input constraints
        u_lbs_tighten = self.env.input_lbs + np.max(self.affine_map(self.Omega_tube, self.K_feedback).V, axis=0)
        u_ubs_tighten = self.env.input_ubs + np.min(self.affine_map(self.Omega_tube, self.K_feedback).V, axis=0)
        self.U_tighten = np.array([u_lbs_tighten, u_ubs_tighten])

        self.tube_bounds_x = self.estimate_bounds_from_polytope(self.Omega_tube)
        self.tube_bounds_u = np.abs(self.K_feedback @ self.tube_bounds_x)

        if self.verbose:
            print(f"Tighten state set X-Ω: {self.X_tighten.V}")
            print(f"Tube size x: {self.tube_bounds_x}")
            print(f"Tube size u: {self.tube_bounds_u}")

        self.setup()
    
    def affine_map(self, poly: pt.Polytope, A: np.ndarray) -> pt.Polytope:
        """Compute the affine image of a polytope under x ↦ A x"""

        assert poly.V is not None, "No poly.V in Polytope! Can not apply affine map."

        V = poly.V  # vertices
        V_new = (A @ V.T).T

        return pt.Polytope(V_new)
        
    def compute_invariant_tube(self, A_cl, lbz, ubz, tol=1e-4, max_iter=10) -> pt.Polytope:
        """
        Compute the robust positive invariant set Omega_tube using Minkowski recursion.

        This implementation uses the `polytope` library's Minkowski sum and affine map.
        """

        # Step 1: Define initial disturbance set D (box)
        dim = len(lbz)

        # H-representation: H x ≤ h
        H_box = np.vstack([np.eye(dim), -np.eye(dim)])
        h_box = np.hstack([ubz, -lbz])

        # Create polytope with both H-rep and V-rep
        D = pt.Polytope(H_box, h_box)

        # Step 2: Initialize Omega := D
        Omega = D

        for i in range(max_iter):
            # Step 3: Apply affine map A_cl to Omega: A_cl * Omega
            A_Omega = self.affine_map(Omega, A_cl)

            # Step 4: Minkowski sum: Omega_next = A_Omega ⊕ D
            Omega_next = A_Omega + D

            # Step 5: Check convergence via bounding box approximation
            bounds_old = self.estimate_bounds_from_polytope(Omega)
            bounds_new = self.estimate_bounds_from_polytope(Omega_next)

            if np.allclose(bounds_old, bounds_new, atol=tol):
                return Omega_next  # Return as vertices

            Omega = Omega_next

        return Omega  # Max iteration reached, return current estimate

    def estimate_bounds_from_polytope(self, poly: pt.Polytope):
        """Estimate box bounds from polytope vertices (axis-aligned)."""
        vertices = poly.V
        return np.max(np.abs(vertices), axis=0)

    def setup(self) -> None:

        ## Model
        # Set up Acados model
        model = AcadosModel()
        model.name = self.name

        # Define model: x_dot = f(x, u)
        model.x = self.dynamics.states
        model.u = self.dynamics.inputs
        model.f_expl_expr = ca.vertcat(self.dynamics.dynamics_function(self.dynamics.states, self.dynamics.inputs))
        model.f_impl_expr = None # no needed, we already have the explicit model

        ## Optimal control problem
        # Set up Acados OCP
        ocp = AcadosOcp()
        ocp.model = model # link to the model (class: AcadosModel)
        ocp.dims.N = self.N  # prediction horizon
        ocp.solver_options.tf = self.N * self.dt  # total prediction time
        ocp.solver_options.qp_solver = "FULL_CONDENSING_HPIPM" # Partially condensing interior-point method
        ocp.solver_options.integrator_type = "ERK" # explicit Runge-Kutta
        ocp.solver_options.nlp_solver_type = "SQP" # sequential quadratic programming

        # Set up cost function
        ocp.cost.cost_type = "LINEAR_LS"
        ocp.cost.cost_type_e = "LINEAR_LS"
        ocp.cost.W = np.block([
            [self.Q, np.zeros((self.dim_states, self.dim_inputs))],
            [np.zeros((self.dim_inputs, self.dim_states)), self.R],
        ])
        ocp.cost.W_e = self.Qf

        # Set up mapping from QP to OCP
        # Define output matrix for non-terminal state
        ocp.cost.Vx = np.block([
            [np.eye(self.dim_states)],
            [np.zeros((self.dim_inputs, self.dim_states))]
        ])
        # Define breakthrough matrix for non-terminal state
        ocp.cost.Vu = np.block([
            [np.zeros((self.dim_states, self.dim_inputs))],
            [np.eye(self.dim_inputs)]
        ])
        # Define output matrix for terminal state
        ocp.cost.Vx_e = np.eye(self.dim_states)

        # Initialize reference of task (stabilization)
        ocp.cost.yref = np.concatenate((self.target_state, np.zeros(self.dim_inputs)))
        ocp.cost.yref_e = self.target_state

        # Input constraints
        ocp.constraints.idxbu = np.arange(self.dim_inputs)

        if self.env.input_lbs is None:
            ocp.constraints.lbu = np.full(self.dim_inputs, -1e6)
        else:
            ocp.constraints.lbu = self.U_tighten[0]

        if self.env.input_ubs is None:
            ocp.constraints.ubu = np.full(self.dim_inputs, 1e6)
        else:
            ocp.constraints.ubu = self.U_tighten[1]

        # Expand initial state constraints (not here, do online)
        # Add Omega constraints on initial state x0: A x0 <= b
        ocp.dims.nh_0 = self.Omega_tube.A.shape[0]
        ocp.model.con_h_expr_0 = ca.mtimes(self.Omega_tube.A, ocp.model.x)
        ocp.constraints.lh_0 = -1e6 * np.ones(self.Omega_tube.A.shape[0])
        ocp.constraints.uh_0 = 1e6 * np.ones(self.Omega_tube.A.shape[0])  # placeholder

        # Expand tighten state constraints 
        ocp.dims.nh = self.X_tighten.A.shape[0]
        ocp.dims.nh_e = self.X_tighten.A.shape[0]
        ocp.model.con_h_expr = ca.mtimes(self.X_tighten.A, ocp.model.x)
        ocp.model.con_h_expr_e = ca.mtimes(self.X_tighten.A, ocp.model.x)
        ocp.constraints.lh = -1e6 * np.ones(self.X_tighten.A.shape[0])
        ocp.constraints.lh_e = -1e6 * np.ones(self.X_tighten.A.shape[0])
        ocp.constraints.uh = self.X_tighten.b.flatten()
        ocp.constraints.uh_e = self.X_tighten.b.flatten()

        # Recreate solver with tightened constraints
        self.ocp = ocp
        self.solver = AcadosOcpSolver(self.ocp, json_file=f"{self.name}.json", generate=True)
        
        if self.verbose:
            print("Tube-based MPC setup with constraint tightening completed.")

    @check_input_constraints
    def compute_action(self, current_state: np.ndarray, current_time) -> np.ndarray:

        # Set upper limit of convex set equality constraint on target step to be 0
        lh_dynamic = self.Omega_tube.A @ current_state + self.Omega_tube.b.flatten()
        self.solver.constraints_set(0, "uh", lh_dynamic)

        status = self.solver.solve()

        x_nominal = self.solver.get(0, "x")
        u_nominal = self.solver.get(0, "u")

        # Apply tube feedback control
        u_real = u_nominal + self.K_feedback @ (current_state - x_nominal)

        if self.verbose:
            print("Current state:", current_state)
            print("Nominal state:", x_nominal)
            print("Nominal input:", u_nominal)
            print("Tube-corrected input:", u_real)

        # Also return nominal predictions
        x_pred = np.zeros((self.N + 1, self.dim_states))
        u_pred = np.zeros((self.N, self.dim_inputs))
        for i in range(self.N + 1):
            x_pred[i, :] = self.solver.get(i, "x")
            if i < self.N:
                u_pred[i, :] = self.solver.get(i, "u")

        return u_real, x_pred, u_pred, u_nominal

    def plot_robust_invariant_set(self, lbz, ubz, tol=1e-4, max_iter=10):
        
        """
        Plot the 2D invariant set (Ω). Assumes 2D state space.
        """

        # Plot
        plt.figure(figsize=(6, 6))

        def plot_polytope(Omega: pt.Polytope, label=None):
            Omega_vertices = Omega.V
            assert Omega_vertices.shape[1] == 2, "Only 2D invariant sets are supported."

            # Convex hull of the Omega polytope
            hull = ConvexHull(Omega_vertices)
            hull_pts = Omega_vertices[hull.vertices]
            hull_pts = np.vstack([hull_pts, hull_pts[0]])

            plt.fill(hull_pts[:, 0], hull_pts[:, 1], color='red', alpha=0.1, label=label)
            plt.plot(hull_pts[:, 0], hull_pts[:, 1], color='red', linewidth=2)

        # Step 1: Define initial disturbance set D (box)
        dim = len(ubz)

        # H-representation: H x ≤ h
        H_box = np.vstack([np.eye(dim), -np.eye(dim)])
        h_box = np.hstack([ubz, -lbz])

        # Create polytope with both H-rep and V-rep
        D = pt.Polytope(H_box, h_box)

        # Step 2: Initialize Omega := D
        Omega = D
        plot_polytope(Omega, label = "Ω")

        A_cl = self.A + self.B @ self.K_feedback

        for i in range(max_iter):
            # Step 3: Apply affine map A_cl to Omega: A_cl * Omega
            A_Omega = self.affine_map(Omega, A_cl)

            # Step 4: Minkowski sum: Omega_next = A_Omega ⊕ D
            Omega_next = A_Omega + D

            # Step 5: Check convergence via bounding box approximation
            bounds_old = self.estimate_bounds_from_polytope(Omega)
            bounds_new = self.estimate_bounds_from_polytope(Omega_next)

            if np.allclose(bounds_old, bounds_new, atol=tol):
                return Omega_next  # Return as vertices

            Omega = Omega_next
            plot_polytope(Omega)

        plt.title("Robust Invariant Set Ω")
        plt.xlabel("Position p")
        plt.ylabel("Velocity v")
        plt.axis('equal')
        plt.grid(True)
        plt.legend()
        plt.show()

<br>

#### **Example 2: Robust Invariant Set and Phase Portrait for RMPC**  

One of the key features that distinguishes Robust MPC (RMPC) from standard MPC is its explicit consideration of worst-case disturbances in the control design. To ensure robust constraint satisfaction under all allowable disturbances, RMPC requires the computation of a **Robust Invariant Set (RIS)** — a set of states from which the system can be kept within constraints, regardless of disturbance realizations.

In this section, we present how the RIS can be recursively constructed from the original disturbance set using set operations. This forms a fundamental step unique to RMPC, as it quantifies the margin of robustness and is later used to tighten constraints and guarantee recursive feasibility.

Finally, we visualize the performance of RMPC in the phase portrait, comparing the actual trajectory with the predicted nominal states and illustrating how the system safely remains within the robust tube defined by the RIS.

In [ ]:
np.random.seed(8042)

In [ ]:
# Define profile of slope, the initial / target state
case = 1 # 1 or 2
initial_position = -0.5
initial_velocity = 0.0
target_position = 0.6
target_velocity = 0.0

# Define the physical boundary condition
state_lbs = np.array([-2.0, -4.0])
state_ubs = np.array([2.0, 4.0])
input_lbs = -8.0
input_ubs = 8.0

# Note: the value here in upper and lower bounds are defined for continuous time, 
#       but the dynamics is discrete, so it will be divided by the frequency in simulator automatically
disturbance_lbs = np.array([-0.05, -0.1])
disturbance_ubs = np.array([0.05, 0.1])

# Instantiate class 'Env'
env = Env(case, np.array([initial_position, initial_velocity]), np.array([target_position, target_velocity]),
          state_lbs=state_lbs, state_ubs=state_ubs, input_lbs=input_lbs, input_ubs=input_ubs,
          disturbance_lbs=disturbance_lbs, disturbance_ubs=disturbance_ubs)

# Instantiate class 'Dynamics'
dynamics = Dynamics(env)

# Define weight matrix in stage and terminal cost and the horizon for MPC 
Q = np.diag([1, 1])
R = np.array([[0.1]])
Qf = Q

# Define parameters of simulation
freq = 10 # controll frequency
t_terminal = 8 # time length of simulation

N = 30


In [ ]:
# Define the RMPC controller
controller_rmpc = LinearRMPCController(env, dynamics, Q, R, Qf, freq, N, max_iter=10, name='LinearRMPC_example', verbose=False)
controller_rmpc.plot_robust_invariant_set(disturbance_lbs, disturbance_ubs)

# Instantiate the simulator, run the simulation, and plot the results
simulator_rmpc = Simulator(dynamics, controller_rmpc, env, 1/freq, t_terminal)
simulator_rmpc.run_simulation()

# Instantiate the visualizer, and display the plottings and animation
visualizer_rmpc = Visualizer(simulator_rmpc)
visualizer_rmpc.display_phase_portrait()

#### **Results Analysis**

The first figure illustrates the iterative computation of the robust invariant set $\Omega_{\text{tube}}$, starting from the initial disturbance set $\mathcal{D}$. Each contour represents one iteration of the recursion $\Omega_{k+1} = \boldsymbol{A}_{\text{cl}} \Omega_k \oplus \mathcal{D}$, gradually expanding until convergence. The final set captures all possible closed-loop state deviations caused by bounded disturbances. This invariant set serves as the cross-sectional shape of the robust tube used in RMPC for constraint tightening.

In the second figure, we visualize the performance of the RMPC controller in the phase portrait, showing the real system trajectory (blue) alongside the nominal trajectory (black dashed). The red tube, centered around the nominal trajectory, represents the translated invariant set $\Omega_{\text{tube}}$ at each time step. It is evident that the real trajectory remains entirely within the robust tube, confirming that RMPC successfully maintains constraint satisfaction under additive disturbances and ensures robust stability throughout the trajectory.

<br>

#### **Example 3: Simulation and Visualization**  

Here, we instantiate the previously implemented RMPC class and apply it to a concrete control problem to evaluate its performance.

Suppose the original discrete-time dynamics are given by:

$$
\boldsymbol{x}_{k+1} = \boldsymbol{A} \boldsymbol{x}_{k} + \boldsymbol{B} \boldsymbol{u}_{k},
$$

We now consider the presence of an additive disturbance $\boldsymbol{w}_k \in \mathcal{D}$, leading to the following disturbed dynamics:

$$
\boldsymbol{x}_{k+1} = \boldsymbol{A} \boldsymbol{x}_{k} + \boldsymbol{B} \boldsymbol{u}_{k} + \boxed{ \boldsymbol{w}_{k} },
$$

where $\boldsymbol{w}_k$ is a random disturbance uniformly distributed within the bounded set $\mathcal{D}=[\boldsymbol{\underline{w}}, \boldsymbol{\overline{w}}]$.

We us the following parameter set as testbed:  

   - case: 1 (linear case)
   
   - initial state: $\boldsymbol{x}_0 = [-0.5, 0.0]^T$

   - target state: $\boldsymbol{x}_T = [0.6, 0.0]^T$

   - state constraints: $ \mathcal{X}_1 = [-2.0, 2.0]$, $ \mathcal{X}_2 = [-4.0, 4.0]$

   - input constraints: $ \mathcal{U} = [-8.0, 8.0]$

   - disturbances bounds: $ \mathcal{D}_1 = [-0.4, 0.4]$, $ \mathcal{D}_2 = [-0.8, 0.8]$

   - weight in cost: $\bm{Q} = \text{diag}([1, 1])$, $\bm{R} = [0.1]$

   - horizon $N = 30$

   - control frequency $f = 10$

We apply both the previously implemented linear MPC and robust MPC controllers to this scenario, run simulations, and observe the resulting behavior.

In [ ]:
# Define profile of slope, the initial / target state
case = 1 # 1 or 2
initial_position = -0.5
initial_velocity = 0.0
target_position = 0.6
target_velocity = 0.0

# Define the physical boundary condition
state_lbs = np.array([-2.0, -4.0])
state_ubs = np.array([2.0, 4.0])
input_lbs = -8.0
input_ubs = 8.0

# Note: the value here in upper and lower bounds are defined for continuous time, 
#       but the dynamics is discrete, so it will be divided by the frequency in simulator automatically
disturbance_lbs = np.array([-0.4, -0.8])
disturbance_ubs = np.array([0.4, 0.8])

# Instantiate class 'Env'
# Arguments (without constraints): 
#   1) case: $n \in [1, 2, 3, 4]$, type: int
#   2) initial state: x_0 = [p_0, v_0], type: np.array
#   3) terminal state: x_T = [p_T, v_T], type: np.array
env = Env(case, np.array([initial_position, initial_velocity]), np.array([target_position, target_velocity]),
          state_lbs=state_lbs, state_ubs=state_ubs, input_lbs=input_lbs, input_ubs=input_ubs,
          disturbance_lbs=disturbance_lbs, disturbance_ubs=disturbance_ubs)
#env.test_env() #  shape of slope (left side) and theta curve (right side) 

# Instantiate class 'Dynamics'
# Arguments: 
#   1) an object of class `Env`, type: Env  
dynamics = Dynamics(env)

In [ ]:
# Define weight matrix in stage and terminal cost and the horizon for MPC 
Q = np.diag([1, 1])
R = np.array([[0.1]])
Qf = Q

N = 30

# Define parameters of simulation
freq = 10 # controll frequency
t_terminal = 8 # time length of simulation

In [ ]:
## LMPC
# Define the MPC controller
controller_mpc = MPCController(env, dynamics, Q, R, Qf, freq, N, name='MPC_baseline', verbose=False)

# Instantiate the simulator, run the simulation, and plot the results
simulator_mpc = Simulator(dynamics, controller_mpc, env, 1/freq, t_terminal)
simulator_mpc.run_simulation()


## RMPC
# Define the RMPC controller
controller_rmpc = LinearRMPCController(env, dynamics, Q, R, Qf, freq, N, max_iter=100, name='RMPC', verbose=False)

# Instantiate the simulator, run the simulation, and plot the results
simulator_rmpc = Simulator(dynamics, controller_rmpc, env, 1/freq, t_terminal)
simulator_rmpc.run_simulation()

# Instantiate the visualizer, and display the plottings and animation
visualizer_rmpc = Visualizer(simulator_rmpc)
visualizer_rmpc.display_contrast_plots("Comparison between the linear MPC and robust MPC under disturbanced scenario", simulator_mpc)
visualizer_rmpc.display_contrast_animation_same(simulator_mpc)


#### **Results Analysis**

The figure above compares the performance of a standard linear MPC controller and a RMPC controller under a disturbed scenario. Although both controllers successfully guide the system toward the target, key differences emerge in how each handles uncertainty.

In the position and velocity plots, we observe that the RMPC trajectory (blue) is generally more conservative but smoother compared to the baseline MPC (red dashed). This is a direct consequence of RMPC’s tube-based formulation, which anticipates worst-case disturbances and proactively tightens constraints using the robust invariant set (RIS).

On the input plot, RMPC tends to inject slightly more conservative control early on to preempt deviation, whereas linear MPC only reacts to observed state errors. Despite similar nominal trajectories, the RMPC ensures that the real trajectory stays within a guaranteed safety tube — something the baseline controller cannot ensure without explicit disturbance modeling.

This contrast also highlights a key insight: **robustness is not simply about reacting to disturbance — it is about proactively embedding worst-case awareness into the optimization structure itself.**



<blockquote style="padding-top: 20px; padding-bottom: 10px;">

##### **🔍 Hands-on Exploration: Improve the Robustness of Normal MPC**

Could we make a linear MPC controller behave more robustly without redesigning it as RMPC? Try it out yourself!

*Hint: think about the three key hyperparameters we introduced in the first part that affecting the MPC's performance most. Which one we should choose?*
